In [1]:
import awswrangler as wr
import pandas as pd
import urllib.parse
import os

# Temporary AWS settings to be set as OS variable in AWS Lambda
os_input_s3_cleansed_layer = os.environ['s3_cleansed_layer']
os_input_glue_catalog_db_name = os.environ['glue_catalog_db_name']
os_input_glue_catalog_table_name = os.environ['glue_catalog_table_name']
os_input_write_data_operation = os.environ['write_data_operation']

def lambda_handler(event, context):
    # getting object from the event and showing its content type
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    
    try:
        # Creating df from content
        df_json = wr.s3.read_json(f's3://{bucket}/{key}')
        
        # Extracting required columns
        df_extract = pd.json_normalize(df_json['items'])
        
        # Writing to s3
        wr_s3_response = wr.s3.to_parquet(df = df_extract,
                                 path = os_input_s3_cleansed_layer,
                                 dataset = True,
                                 database = os_input_glue_catalog_db_name,
                                 table = os_input_glue_catalog_table_name,
                                 mode = os_input_write_data_operation
                                )
        return wr_s3_response
    except Exception as e:
        print(e)
        print(f'Error getting object {key} from the {bucket}')
        raise e

In [ ]:
import awswrangler as wr
import pandas as pd
import urllib.parse
import os

# Temporary AWS settings to be set as OS variable in AWS Lambda
os_input_s3_cleansed_layer = os.environ('s3_cleansed_layer')
os_input_glue_catalog_db_name = os.environ('glue_catalog_db_name')
os_input_glue_catalog_table_name = os.environ('glue_catalog_table_name')
os_input_write_data_operation = os.environ('write_data_operation')

def lambda_handler(event, context):
    # getting object from the event and showing its content type
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    
    try:
        # Creating df from content
        df_json = wr.s3.read_json('s3://{}/{}'.format(bucket,key))
        
        # Extracting required columns
        df_extract = pd.json_normalize(df_json['items'])
        
        # Writing to s3
        wr_s3_response = wr.s3.to_parquet(df = df_extract,
                                 path = os_input_s3_cleansed_layer,
                                 dataset = True,
                                 database = os_input_glue_catalog_db_name,
                                 table = os_input_glue_catalog_table_name,
                                 mode = os_input_write_data_operation
                                )
        return wr_s3_response
    except Exception as e:
        print(e)
        print(f'Error getting object {key} from the {bucket}')
        raise e


In [ ]:
Key
	
Value
glue_catalog_db_name	db_yt_cleaned
glue_catalog_table_name	yt_json_cleaned
s3_cleansed_layer	s3://yt-cleaned-data/youtube
write_data_operation	append